In [1]:
import json
from tqdm import tqdm

In [2]:
def convert_to_sharegpt(src_file,tgt_file):
    with open(src_file,mode="r") as src,open(tgt_file,mode="w") as tgt:
        conversations = json.load(src)
        target_list = []
        print("converting into sharegpt format")
        
        for id,conversation in tqdm(conversations.items()):
            self_report = {"from":"user",
                           "value":conversation["self_report"]}
            previous_sentence = self_report
            # if previous speaker and current speaker are same, merge the sentences
            
            conv = {
                # "id": id,
                "conversations":[],
                "system":"你是一个智能医疗诊断助手，请根据用户输入的症状描述，解答用户的问题或给出医学建议，推动对话直至得出医学诊断结论，如果信息不足，请要求用户提供必要的信息。",
            }
            conv["conversations"].append(previous_sentence)
            
            for item in  conversation["dialogue"]:
                role = item["speaker"]
                value = item["sentence"]
                if ("user" if role=="患者" else "assistant") == previous_sentence["from"]:
                    if previous_sentence["value"].endswith(tuple(["，","。","？","！","~","；","、"])):
                        previous_sentence["value"] = previous_sentence["value"]+" "+value
                    else:
                        previous_sentence["value"] = previous_sentence["value"]+"。"+value
                    # print("current role == previous role:"+ role,"merged value="+previous_sentence["value"])
                    conv["conversations"][-1] = previous_sentence
                else:
                    sentence =  { "from": "user" if role=="患者" else "assistant",
                      "value": value}
                    conv["conversations"].append(sentence)
                    previous_sentence = sentence
            #the last response should be by assistant,otherwise, append one sentence
            if(conv["conversations"][-1]["from"]=="user"):
                conv["conversations"].append({
                    "from":"assistant",
                    "value":"祝您早日康复，再见"
                })
            target_list.append(conv)
            # print(target_list)
            
            
        print("print into target json file as training materials")
        json.dump(obj=target_list,fp=tgt,ensure_ascii=False,indent=4)
        print(f"writing to {tgt} completed")
    
print("definition done")

definition done


In [3]:
convert_to_sharegpt(src_file="./NER/IMCS-V2_train.json",tgt_file="./NER/IMCS-V2_train_conversation.json")
convert_to_sharegpt(src_file="./NER/IMCS-V2_dev.json",tgt_file="./NER/IMCS-V2_dev_conversation.json")

converting into sharegpt format


100%|██████████| 2472/2472 [00:00<00:00, 38243.11it/s]

print into target json file as training materials


writing to <_io.TextIOWrapper name='./NER/IMCS-V2_train_conversation.json' mode='w' encoding='UTF-8'> completed
converting into sharegpt format


100%|██████████| 833/833 [00:00<00:00, 50994.02it/s]

print into target json file as training materials
writing to <_io.TextIOWrapper name='./NER/IMCS-V2_dev_conversation.json' mode='w' encoding='UTF-8'> completed


### 加入dataset_info.json
```json
  "IMCS-V2_train_conversation": {
    "file_name": "IMCS-V2_train_conversation.json",
    "formatting": "sharegpt",
    "columns": {
      "messages": "conversations"
    },
    "tags": {
      "role_tag": "from",
      "content_tag": "value",
      "user_tag": "user",
      "assistant_tag": "assistant"
    }
  },
```

### 测试结果 0.5B三轮全参数训练的结果
```
[INFO|2025-01-08 16:30:30] llamafactory.model.model_utils.attention:157 >> Using torch SDPA for faster training and inference.
[INFO|2025-01-08 16:30:30] llamafactory.model.loader:157 >> all params: 494,032,768

        Average: 40.83                                                                                                                                
           STEM: 35.165%|██████████████████████▍                                                                    | 52/211 [00:00<00:00, 256.96it/s]
Social Sciences: 42.03
     Humanities: 41.46
          Other: 43.71
```

### 测试结果 7B十二轮LoRA训练结果
```
        Average: 77.59                                                                                                                                                                 
           STEM: 71.51
Social Sciences: 78.04
     Humanities: 80.07
          Other: 80.17
```

### 测试结果 7B预训练模型
```
        Average: 80.20                                                                                                                                
           STEM: 73.25
Social Sciences: 80.28
     Humanities: 83.29
          Other: 83.51
```